In [ ]:
import pandas as pd
pd.Series

import math
import csv
from datetime import datetime

import numpy as np
import scipy as sc

import statsmodels
import sklearn
from sklearn import preprocessing

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 25, 20
import time

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from matplotlib import pyplot

from keras.regularizers import l1
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm

# import BatchNormalization
from keras.layers.normalization import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.regularizers import l2

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score 

### Load file, print info and select columns

In [ ]:
#function to load files
def load_file(filepath):
    df = pd.read_csv(filepath, sep='\t', index_col=0, parse_dates=True)
    df = df.sort_index()
    #we can check that this 2 columns are equal, so we can drop one
    #any(df['SALE_AMOUNT_BEFORE_CANCELLATIONS'] != df['SALE_AMOUNT_AFTER_CANCELLATIONS'])
    df = df.drop(['SALE_AMOUNT_AFTER_CANCELLATIONS'], axis=1)
#...
    return df.astype('float32')

def load_file1(filepath):
    df = pd.read_csv(filepath, sep=',', index_col=0, parse_dates=True)
    df = df.sort_index()
    return df.astype('float32')


#function to create a new df with selected columns
def create_small_df(df, columns):
    small_df = df.copy()
    small_df = small_df[columns]
    return small_df

#function to print inf about Data
def print_info_df(df, print_columns = False):
    #Count period
    d1 = df.index[0]
    d2 = df.index[-1]
    delta = d2 - d1
    print('Number of days is ' + str(delta.days) + ' from ' + str(d1) + ' to '+ str(d2))
    print('The shape of the data: %d*%d' %(df.shape[0],df.shape[1]))
    print('Check for Nan values: %s'%(df.isnull().values.any()))
    if (print_columns == True):
        print(list(df.columns))
    else:
        print('Number of columns: %d'%(df.shape[1]))

In [ ]:
df = load_file('..')
print_info_df(df, False)
feature = 'SALE_AMOUNT'

### try new features

In [ ]:
df_hol = load_file1('../..')
df_hol_sel = load_file1('../..')
df_sel = load_file1('../..')

In [ ]:
df_hol_train = df_hol.head(1642)
df_hol_sel_train = df_hol.head(1642)
df_sel_train = df_hol.head(1642)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    epsilon = 0.000001
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

def root_mean_square_error(y_true, y_pred):
    #y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(mean_squared_error(y_true, y_pred))

def f_relative_rmse(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    rmse = root_mean_square_error(y_true, y_pred)
    return (rmse / np.mean(y_true))*100

#def f_smape(A, F):
#    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

def f_smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    epsilon = 0.1
    summ = np.maximum(np.abs(y_true) + np.abs(y_pred) + epsilon, 0.5 + epsilon)
    smape = np.abs(y_pred - y_true) / summ * 2.0
    return np.mean(smape)

def summarize_results(scores):
    #print(scores)
    m, s = np.mean(scores), np.std(scores)
    #print('Mean %.4f (+/- %.4f)' % (m,s))
    return m,s

### LSTM

In [ ]:
#LSTM Data Preparation
# convert series to supervised learning

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

def preprocessing_data(data, n, s_columns):
    # normalize features
    scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
    scaled = scaler.fit_transform(data)
    # frame as supervised learning
    reframed = series_to_supervised(scaled, n, 1)
    # drop columns we don't want to predict
    columns_to_drop = list(range(-s_columns+1,0))
    reframed.drop(reframed.columns[columns_to_drop], axis=1, inplace=True)
    values = reframed.values
    return scaler, values

def reshape_ts_data(n_steps, s_columns, train_X, train_y, test_X, test_y):
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], n_steps, s_columns))
    test_X = test_X.reshape((test_X.shape[0], n_steps, s_columns))
    return train_X, train_y, test_X, test_y

In [ ]:
# def time_split_data(values, n_split=9):
#     index=1
#     plt.figure(1)
#     X = values[:, 1:]
#     y = values[:, 0]
#     tscv = TimeSeriesSplit(n_splits=n_split)
#     for train_index, test_index in tscv.split(X):
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]
#         print('Train X: (%d,%d), Test X: (%d,%d)' % (X_train.shape[0], X_train.shape[1], X_test.shape[0], X_test.shape[1]))
#         print('Train y: (%d), Test y: (%d)' % (y_train.shape[0], y_test.shape[0]))
#         plt.subplot(910 + index)
#         plt.plot(y_train)
#         plt.plot([None for i in y_train] + [x for x in y_test])
#         index += 1
#     plt.show()

In [ ]:
# values = df.values
# time_split_data(values, 9)

In [ ]:
def define_fit_lstm(train_X, train_y, test_X, test_y, n_steps, scaler, s_columns):
    s = s_columns
    n_input = n_steps*s
    model = Sequential()
    model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='Adagrad', metrics=['mse', 'mae', 'mape'])
    history = model.fit(train_X, train_y, epochs=100, verbose=0, batch_size=n_input, shuffle=False,
                        validation_data=(test_X, test_y))
    # fit network
    ###history = model.fit(train_X, train_y, epochs=100, verbose=0, batch_size=n_input, shuffle=False)
    # plot history 
    plt.subplot(2, 1, 1)
    plt.plot(history.history['loss'], label='loss_MAE', lw=2)
    plt.plot(history.history['val_loss'], label='val_loss_MAE', lw=2)
    plt.title('Training and Validation Loss')
    plt.legend(prop={'size': 15})
    plt.grid(True)
    plt.show()
    
    # evaluate model
    results = model.evaluate(test_X, test_y, verbose=0)
    loss, mse, mae, mape = results 
    print('loss=%.3f, mse=%.3f, mae=%.3f, mape=%.3f' %(loss, mse, mae, mape)) 
    print ('History results: ')
    print('Loss: %.3f - %.3f' % (history.history['loss'][0],history.history['loss'][-1]))
    print('Validation Loss: %.3f - %.3f' % (history.history['val_loss'][0],history.history['val_loss'][-1]))
   
    # make a prediction
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], n_steps*s_columns))
    # invert scaling for forecast
    inv_yhat = np.concatenate((yhat, test_X[:, 1-s:]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    # invert scaling for actual
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = np.concatenate((test_y, test_X[:, 1-s:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]
    
    # calculate RMSE
    rmse = root_mean_square_error(inv_y, inv_yhat)
    rel_rmse = f_relative_rmse(inv_y, inv_yhat)
    # recalculate MAPE (results are the same actually)
    mape = mean_absolute_percentage_error(inv_y, inv_yhat)
    #print('Test RMSE: %.3f' % rmse)
    #print('Test MAPE: %.3f' % mape)
    plt.subplot(2, 1, 1)
    plt.plot(inv_yhat, label='yhat', linestyle='--', lw=2)
    plt.plot(inv_y, label='y', lw=2)
    plt.title('Observed and Predicted Values')
    plt.legend(prop={'size': 15})
    plt.grid(True)
    plt.show() 
    
    plt.subplot(2, 1, 1)
    plt.plot(inv_yhat[:30], label='yhat', linestyle='--', lw=2)
    plt.plot(inv_y[:30], label='y', lw=2)
    plt.title('Observed and Predicted Values')
    plt.legend(prop={'size': 15})
    plt.grid(True)
    plt.show() 
    
    return loss, rmse, mape, rel_rmse

def run_time_split_data(values, n_steps, s_columns, scaler, n_split):
    X = values[:, :-1]
    y = values[:, -1]
    #new = np.c_[X, y]  
    k=1
    losses = list()
    rmses = list()
    mapes = list()
    relative_rmses = list()
    
    skip = 1
    
    tscv = TimeSeriesSplit(n_splits=n_split)
    for train_index, test_index in tscv.split(X):
        if (skip > 8):
            print(' ')
            print('------------------------------------%d-------------------------------------------' %(k))
            k+=1
            train_X, test_X = X[train_index], X[test_index]
            train_y, test_y = y[train_index], y[test_index]
            print('Train X: (%d,%d), Test X: (%d,%d)' % (train_X.shape[0], train_X.shape[1],
                                                         test_X.shape[0], test_X.shape[1]))
            print('Train y: (%d), Test y: (%d)' % (train_y.shape[0], test_y.shape[0]))
            new_train_X, new_train_y, new_test_X, new_test_y = reshape_ts_data(n_steps, s_columns, 
                                                                               train_X, train_y, test_X, test_y)
            #Define and fit our LSTM model
            #Make a prediction
            loss, rmse, mape, rel_rmse = define_fit_lstm(new_train_X, new_train_y, new_test_X, new_test_y,
                                               n_steps, scaler, s_columns)
            print('Training Loss: %.3f' % (loss))
            losses.append(loss)
            print('Test RMSE: %.3f' % (rmse))
            rmses.append(rmse)
            print('Relative RMSE: %.3f' % (rel_rmse))
            relative_rmses.append(rel_rmse)
            print('Test MAPE: %.3f' % (mape))
            mapes.append(mape)
            print('---------------------------------------')
            print('---------------------------------------')
        skip = skip+1
        
    return losses, rmses, mapes, relative_rmses
        
    
def run_model(n_steps, df, n_split=15):
    values = df.values.astype('float32')
    feature_set = list(df.columns)
    s_columns = df.shape[1]
    print('Run model')
    print('Features set:'+ str(feature_set))
    #Preprocessing
    scaler, new_values = preprocessing_data(values, n_steps, s_columns)
    #Run
    losses, rmses, mapes, relative_rmses = run_time_split_data(new_values, n_steps,
                                                               s_columns, scaler, n_split)
    print('--------------------------------------------------------------------------------------------------------')    
    print('Final Average Results: ')
    m,s = summarize_results(losses)
    print('Loss: %.4f (s=%.4f)' % (m,s))
    m,s = summarize_results(rmses)
    print('RMSE: %.4f (s=%.4f)' % (m,s))
    m,s = summarize_results(relative_rmses)
    print('Relative RMSE: %.4f (s=%.4f)' % (m,s))
    m,s = summarize_results(mapes)
    print('MAPE: %.4f (s=%.4f)' % (m,s))

In [ ]:
run_model(4, df)

In [ ]:
# df_hol_train = df_hol.head(1642)
# df_hol_sel_train = df_hol.head(1642)
# df_sel_train = df_hol.head(1642)

In [ ]:
#engine

In [ ]:
run_model(2, df_hol_train)

In [ ]:
run_model(3, df_hol_train)

In [ ]:
run_model(4, df_hol_train)

In [ ]:
run_model(5, df_hol_train)

In [ ]:
run_model(6, df_hol_train)

In [ ]:
run_model(9, df_hol_train)

### sel_eng

In [ ]:
run_model(2, df_hol_sel_train)

In [ ]:
run_model(3, df_hol_sel_train)

In [ ]:
run_model(4, df_hol_sel_train)

In [ ]:
run_model(5, df_hol_sel_train)

In [ ]:
run_model(6, df_hol_sel_train)

In [ ]:
run_model(7, df_hol_sel_train)

In [ ]:
# with selected only

In [ ]:
run_model(2, df_sel_train)

In [ ]:
run_model(3, df_sel_train)

In [ ]:
run_model(4, df_sel_train)

In [ ]:
run_model(5, df_sel_train)

In [ ]:
run_model(6, df_sel_train)

In [ ]:
run_model(7, df_sel_train)

In [ ]:
run_model(8, df_sel_train)